In [3]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
CALCULATED_DRAW_DOWNS = dataiku.Dataset("CALCULATED_DRAW_DOWNS")
CALCULATED_DRAW_DOWNS_df = CALCULATED_DRAW_DOWNS.get_dataframe()

CALCULATED_DRAW_UPS = dataiku.Dataset("CALCULATED_DRAW_UPS")
CALCULATED_DRAW_UPS_df = CALCULATED_DRAW_UPS.get_dataframe()

COMMON_WORDS = dataiku.Dataset("NAFCUSTOMER_COMMON_WORDS_IN_NAMES")
COMMON_WORDS_df = COMMON_WORDS.get_dataframe()

In [2]:
def date_tz_naive(pd_s):
    return pd.to_datetime(pd_s).apply(lambda x:x.tz_localize(None))

In [5]:
# consider customers with 10 cards or more

df_down_full = CALCULATED_DRAW_DOWNS_df[CALCULATED_DRAW_DOWNS_df.ACTIVE_CARD_MAX>10].copy()
df_up_full = CALCULATED_DRAW_UPS_df[CALCULATED_DRAW_UPS_df.ACTIVE_CARD_MAX>10].copy()

df_down_full.DRAW_DOWN_DATE = date_tz_naive(df_down_full['DRAW_DOWN_DATE'])
df_up_full.DRAW_UP_DATE = date_tz_naive(df_up_full['DRAW_UP_DATE'])

df_up_full.dropna(subset=['DRAW_UP_DATE'], inplace=True)

df_common = COMMON_WORDS_df

df_down_full.sort_values(['CUSTOMER'], inplace=True)
df_up_full.sort_values(['CUSTOMER'], inplace=True)
df_common.sort_values(['WORD'], inplace=True)

print(len(df_down_full), "draw downs full")
print(len(df_up_full), "draw ups full")
print(len(df_common), "common words")

16916 draw downs full
32961 draw ups full
2173 common words


In [6]:
import string

_common_words = df_common.WORD.unique()
print(len(_common_words), "screening against common words")

class Draw_Down_Customer:

    def __init__(self, name, draw_down_date, active_card_max):

        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max

        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        self.DAYS_DIFFERENCE = []
        self.DRAW_UP_DATE = []

        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in _common_words:
                self.WORD_LIST.append(w)

    def Match_Draw_Up_Customer(self, name, draw_up_date, active_card_max):

        if (self.CUSTOMER == name):
            # exact match, already captured
            return

        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()

        check_list = []
        for w in f:
            if (w not in _common_words) and (len(w)>1) and (not w.isnumeric()):
                check_list.append(w)

        percent_diff = round((abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2)),2)

        #date_format = "%Y-%m-%d"
        #d1_date = datetime.strptime(draw_up_date.astype(str), date_format)
        #d2_date = datetime.strptime(self.DRAW_DOWN_DATE.astype(str), date_format)

        delta_between_drop_and_rise = round(abs((draw_up_date-self.DRAW_DOWN_DATE).days)/30.,0)

        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:

                    if not name in(self.MATCHING_CUSTOMERS) and(delta_between_drop_and_rise<=4)and(percent_diff<=0.5) :
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        self.DAYS_DIFFERENCE.append(delta_between_drop_and_rise)
                        self.DRAW_UP_DATE.append(draw_up_date)
                        break;

2173 screening against common words


In [7]:
import time

def do_save_log(_matching_process_log_time, _matching_process_log_event):

    df_matching_log = pd.DataFrame(_matching_process_log_time)
    if len(df_matching_log)>0:

        df_matching_log.columns = ['LOG_TIME']
        df_matching_log['LOG_EVENT'] = _matching_process_log_event

        MATCHING_PROCESS_LOG_V_df = df_matching_log
        MATCHING_PROCESS_LOG_V = dataiku.Dataset("MATCHING_PROCESS_LOG_V")
        MATCHING_PROCESS_LOG_V.write_with_schema(MATCHING_PROCESS_LOG_V_df)

        print()

def do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date):

    df_matches = pd.DataFrame(_direct_customer)
    if len(df_matches)>0:

        print()
        print("saving", len(df_matches), "1-1 matching records")
        print()

        df_matches.columns = ['CUSTOMER']
        df_matches["MATCH_CUSTOMER"] = _direct_match
        df_matches["DRAW_UP_DATE"] = _direct_draw_up_date

        MATCHES_1_TO_1_STAGING_V_df = df_matches
        MATCHES_1_TO_1_STAGING_V = dataiku.Dataset("MATCHES_1_TO_1_STAGING_V")
        MATCHES_1_TO_1_STAGING_V.write_with_schema(MATCHES_1_TO_1_STAGING_V_df)

        print()

def do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates):

    df_multiple_matches = pd.DataFrame(_multiple_customer)

    if len(df_multiple_matches)>0:

        print()
        print("saving", len(df_multiple_matches), "1-n matching records")
        print()

        df_multiple_matches.columns = ['CUSTOMER']
        df_multiple_matches["MATCH_CUSTOMER"] = _multiple_matches
        df_multiple_matches["DRAW_UP_DATE"] = _multiple_drop_dates

        MATCHES_1_TO_N_STAGING_V_df = df_multiple_matches
        MATCHES_1_TO_N_STAGING_V = dataiku.Dataset("MATCHES_1_TO_N_STAGING_V")
        MATCHES_1_TO_N_STAGING_V.write_with_schema(MATCHES_1_TO_N_STAGING_V_df)

        print()

In [8]:
from datetime import timedelta

df_down = df_down_full
df_up = df_up_full

_processed_customers = []
verbose = False

_matching_process_log_time = []
_matching_process_log_event = []

_direct_customer = []
_direct_match = []
_direct_draw_up_date = []

_multiple_customer = []
_multiple_matches = []
_multiple_drop_dates = []

_no_match_customer = []

save_every_n = 50
to_save_counter = 0
print_every_n = 25

print(len(df_down), "filtered down rows")
print(len(df_up), "filtered up rows")

_customers = []

t0 = time.time()

for index, row in df_down.iterrows():
    
    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    active_card_max = row['ACTIVE_CARD_MAX']

    c = Draw_Down_Customer(customer, draw_down_date, active_card_max)

    _customers.append(c)

idx = 0

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append(" processing range " + str(len(_customers)) + " Draw Down Customers")
do_save_log(_matching_process_log_time, _matching_process_log_event)

for c in _customers:
    
    idx+=1
    
    date_start = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=-120)
    date_end = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=120)
    
    card_delta = c.ACTIVE_CARD_MAX * 0.5
    card_start = c.ACTIVE_CARD_MAX - card_delta
    card_end = c.ACTIVE_CARD_MAX + card_delta
    
    df_up = df_up_full[(df_up_full.ACTIVE_CARD_MAX>=card_start)&
                   (df_up_full.ACTIVE_CARD_MAX<=card_end)&
                    (df_up_full.DRAW_UP_DATE >= pd.to_datetime(date_start))&
                  (df_up_full.DRAW_UP_DATE <= pd.to_datetime(date_end))]
    
    for index_up, row_up in df_up.iterrows():

        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        active_card_max = row_up['ACTIVE_CARD_MAX']

        c.Match_Draw_Up_Customer(customer, draw_up_date, active_card_max)
        
    if len(c.MATCHING_CUSTOMERS)==1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _direct_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _direct_match.append(c.MATCHING_CUSTOMERS[0])
            _processed_customers.append(c.MATCHING_CUSTOMERS[0])
            _direct_draw_up_date.append(c.DRAW_UP_DATE[0])

            if verbose:
                print()
                print("DIRECT")
                print(c.CUSTOMER, c.WORD_LIST)
                print(c.MATCHING_CUSTOMERS)
                print(c.PERCENT_DIFFERENCE)
                print(c.DAYS_DIFFERENCE)
                print("=====")
                print()

    elif len(c.MATCHING_CUSTOMERS)>1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _multiple_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _multiple_matches.append(c.MATCHING_CUSTOMERS)
            _multiple_drop_dates.append(c.DRAW_UP_DATE)

        if verbose:
            print()
            print("MULTIPLE")
            print(c.CUSTOMER, c.WORD_LIST)
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print("=====")
            print()

    else:

        # could not find a match, remove it from future processing
        _no_match_customer.append(c.CUSTOMER)
        _processed_customers.append(c.CUSTOMER)

    if to_save_counter>=save_every_n:

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("writing datasets to snowflake")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
        do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        to_save_counter = 0
        
    t1 = time.time()
    
    avg_duration = (((t1-t0)/idx)/60.0)
    
    if idx % print_every_n == 0:
        idx_remaining = len(_customers)-idx
        print("processing", idx, "current record:", c.CUSTOMER, ",", idx_remaining, "remaining")
        print(round(avg_duration,2), "avg mins per iteration",  round((avg_duration*idx_remaining)/60,2), "estimated hrs remaining")
        print(len(_direct_customer), "direct match records", len(_multiple_customer), "multiple match records", len(_no_match_customer), "no match records")
        print()
        
_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("writing datasets to snowflake")
do_save_log(_matching_process_log_time, _matching_process_log_event)

do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
do_save_log(_matching_process_log_time, _matching_process_log_event)


16916 filtered down rows
32961 filtered up rows
1 rows successfully written (lOOClm7eoz)

processing 25 current record: 1000058 ALTRA MOTION , 16891 remaining
0.02 avg mins per iteration 4.47 estimated hrs remaining
3 direct match records 0 multiple match records 22 no match records

processing 50 current record: 10369918 CANADA CORPORATION , 16866 remaining
0.01 avg mins per iteration 3.45 estimated hrs remaining
4 direct match records 1 multiple match records 45 no match records

processing 75 current record: 1597 WILLIAMS GAS PIPELINE(05) , 16841 remaining
0.01 avg mins per iteration 3.25 estimated hrs remaining
7 direct match records 2 multiple match records 66 no match records

processing 100 current record: 1763889 ONTARIO INC. , 16816 remaining
0.01 avg mins per iteration 2.91 estimated hrs remaining
8 direct match records 2 multiple match records 90 no match records

processing 125 current record: 18085-GENERAL SUPPLY AND SERV2 , 16791 remaining
0.01 avg mins per iteration 2.75

processing 1025 current record: AFFORDABLE DRAIN SERVICE INC , 15891 remaining
0.01 avg mins per iteration 3.44 estimated hrs remaining
73 direct match records 9 multiple match records 943 no match records

processing 1050 current record: AGRICARE NUTRITION LLC , 15866 remaining
0.01 avg mins per iteration 3.44 estimated hrs remaining
80 direct match records 11 multiple match records 959 no match records

processing 1075 current record: AIR CONDITIONING BY JAY INC , 15841 remaining
0.01 avg mins per iteration 3.43 estimated hrs remaining
83 direct match records 12 multiple match records 980 no match records

processing 1100 current record: AIR TECH OF HOUSTON , 15816 remaining
0.01 avg mins per iteration 3.43 estimated hrs remaining
83 direct match records 12 multiple match records 1005 no match records

processing 1125 current record: AIRWAY OXYGEN INC , 15791 remaining
0.01 avg mins per iteration 3.43 estimated hrs remaining
85 direct match records 13 multiple match records 1027 no m

processing 1950 current record: AUTOBUS OUTAOUAIS INC , 14966 remaining
0.01 avg mins per iteration 3.53 estimated hrs remaining
193 direct match records 35 multiple match records 1722 no match records

processing 1975 current record: AVALON HEALTH CARE , 14941 remaining
0.01 avg mins per iteration 3.54 estimated hrs remaining
197 direct match records 35 multiple match records 1743 no match records

processing 2000 current record: AWS FIBER LLC , 14916 remaining
0.01 avg mins per iteration 3.53 estimated hrs remaining
200 direct match records 35 multiple match records 1765 no match records

processing 2025 current record: B  B PROFESSIONAL PLUMBING INC , 14891 remaining
0.01 avg mins per iteration 3.53 estimated hrs remaining
204 direct match records 35 multiple match records 1786 no match records

processing 2050 current record: B & V MECHANICAL INC , 14866 remaining
0.01 avg mins per iteration 3.53 estimated hrs remaining
205 direct match records 35 multiple match records 1810 no mat

processing 2875 current record: BTG AMERICAS INC 0T25 , 14041 remaining
0.01 avg mins per iteration 3.4 estimated hrs remaining
324 direct match records 64 multiple match records 2486 no match records

processing 2900 current record: BUGSORUS INC , 14016 remaining
0.01 avg mins per iteration 3.4 estimated hrs remaining
325 direct match records 64 multiple match records 2510 no match records

processing 2925 current record: BULLYS TRANSPORT NJ LLC , 13991 remaining
0.01 avg mins per iteration 3.4 estimated hrs remaining
327 direct match records 65 multiple match records 2532 no match records

processing 2950 current record: BUSTED KNUCKLE AUTO , 13966 remaining
0.01 avg mins per iteration 3.4 estimated hrs remaining
330 direct match records 66 multiple match records 2553 no match records

processing 2975 current record: C  S VENDING INC , 13941 remaining
0.01 avg mins per iteration 3.39 estimated hrs remaining
331 direct match records 66 multiple match records 2577 no match records

pro

processing 3800 current record: CITY OF RIFLE COLORADO , 13116 remaining
0.01 avg mins per iteration 3.24 estimated hrs remaining
428 direct match records 82 multiple match records 3289 no match records

processing 3825 current record: CITY TAXICAB AND TRANSFER , 13091 remaining
0.01 avg mins per iteration 3.23 estimated hrs remaining
430 direct match records 82 multiple match records 3312 no match records

processing 3850 current record: CLARITY TELECOM , 13066 remaining
0.01 avg mins per iteration 3.22 estimated hrs remaining
435 direct match records 82 multiple match records 3332 no match records

processing 3875 current record: CLEAN ENERGY COMFORT INC , 13041 remaining
0.01 avg mins per iteration 3.22 estimated hrs remaining
439 direct match records 82 multiple match records 3353 no match records

processing 3900 current record: CLEARWAVE COMMUNICATIONS 5GF1 , 13016 remaining
0.01 avg mins per iteration 3.21 estimated hrs remaining
444 direct match records 82 multiple match record

...

processing 13075 current record: ROBBINS GARDEN CENTER INC , 3841 remaining
0.01 avg mins per iteration 0.96 estimated hrs remaining
1589 direct match records 252 multiple match records 11226 no match records

processing 13100 current record: ROBS AIR CONDITIONING  HEATING INC , 3816 remaining
0.01 avg mins per iteration 0.95 estimated hrs remaining
1594 direct match records 254 multiple match records 11244 no match records

processing 13125 current record: RODENT PEST TECHNOLOGIES INC , 3791 remaining
0.01 avg mins per iteration 0.94 estimated hrs remaining
1595 direct match records 254 multiple match records 11268 no match records

74 rows successfully written (Kdm5WSytEW)


saving 1596 1-1 matching records

1596 rows successfully written (KFiWcYPZd2)


saving 254 1-n matching records

254 rows successfully written (oC4wsDLX0i)

75 rows successfully written (XtQqsJnBip)

processing 13150 current record: RONS TOWING LLC , 3766 remaining
0.01 avg mins per iteration 0.94 estimated hrs r

processing 13975 current record: SLECT LANDSCAPES LLC , 2941 remaining
0.01 avg mins per iteration 0.73 estimated hrs remaining
1711 direct match records 269 multiple match records 11986 no match records

processing 14000 current record: SMITH ACG INCORPORATED , 2916 remaining
0.01 avg mins per iteration 0.73 estimated hrs remaining
1716 direct match records 269 multiple match records 12006 no match records

processing 14025 current record: SOAP SLINGERS WINDOW CLEANING , 2891 remaining
0.01 avg mins per iteration 0.72 estimated hrs remaining
1719 direct match records 269 multiple match records 12028 no match records

processing 14050 current record: SOLCIUS LLC , 2866 remaining
0.01 avg mins per iteration 0.71 estimated hrs remaining
1723 direct match records 269 multiple match records 12049 no match records

processing 14075 current record: SOUND HEALTH MANAGEMENT , 2841 remaining
0.01 avg mins per iteration 0.71 estimated hrs remaining
1725 direct match records 269 multiple match re

processing 14875 current record: TAYLA NIXON SITE DEVELOPERS , 2041 remaining
0.01 avg mins per iteration 0.51 estimated hrs remaining
1830 direct match records 280 multiple match records 12755 no match records

processing 14900 current record: TEACH FOR AMERICA INC , 2016 remaining
0.01 avg mins per iteration 0.5 estimated hrs remaining
1833 direct match records 280 multiple match records 12777 no match records

processing 14925 current record: TEK TRANSPORTATION SERVICES LLC , 1991 remaining
0.01 avg mins per iteration 0.5 estimated hrs remaining
1837 direct match records 280 multiple match records 12798 no match records

processing 14950 current record: TENDER LAWN CARE INC (04)(2) , 1966 remaining
0.01 avg mins per iteration 0.49 estimated hrs remaining
1840 direct match records 281 multiple match records 12819 no match records

processing 14975 current record: TETON MOVERS , 1941 remaining
0.01 avg mins per iteration 0.48 estimated hrs remaining
1845 direct match records 283 multi

processing 15800 current record: U S A GENERAL CONTRACTORS CORP , 1116 remaining
0.01 avg mins per iteration 0.28 estimated hrs remaining
1931 direct match records 299 multiple match records 13559 no match records

processing 15825 current record: UFCW UNION LOCAL 770 , 1091 remaining
0.01 avg mins per iteration 0.27 estimated hrs remaining
1932 direct match records 299 multiple match records 13583 no match records

processing 15850 current record: UNIFOR , 1066 remaining
0.01 avg mins per iteration 0.27 estimated hrs remaining
1932 direct match records 299 multiple match records 13608 no match records

processing 15875 current record: UNITED BUSINESS MAIL INC , 1041 remaining
0.01 avg mins per iteration 0.26 estimated hrs remaining
1932 direct match records 299 multiple match records 13633 no match records

processing 15900 current record: UNITED LAWNSCAPE OF FLORI , 1016 remaining
0.01 avg mins per iteration 0.25 estimated hrs remaining
1935 direct match records 299 multiple match re

processing 16725 current record: WOLFPACK LLC , 191 remaining
0.01 avg mins per iteration 0.05 estimated hrs remaining
2025 direct match records 312 multiple match records 14374 no match records

processing 16750 current record: WPS INC , 166 remaining
0.01 avg mins per iteration 0.04 estimated hrs remaining
2029 direct match records 314 multiple match records 14393 no match records

processing 16775 current record: WYOMING ROOFING & SUPPLY LLC , 141 remaining
0.01 avg mins per iteration 0.04 estimated hrs remaining
2032 direct match records 314 multiple match records 14415 no match records

processing 16800 current record: XTREME LAWN CARE , 116 remaining
0.01 avg mins per iteration 0.03 estimated hrs remaining
2033 direct match records 314 multiple match records 14439 no match records

94 rows successfully written (hzZ2xYLwRW)


saving 2036 1-1 matching records

2036 rows successfully written (hIicPYbRFe)


saving 314 1-n matching records

314 rows successfully written (VdP24O1IjL)

